In [20]:
import json
import os
from pathlib import Path

In [44]:
import matplotlib.pyplot as plt
import pandas as pd
import scanpy as sc
import typer
from sklearn.cluster import KMeans
from supabase import create_client
from dotenv import load_dotenv

In [8]:
from cosilico_py.client.client import CosilicoClient
from cosilico_py.config import get_config
from cosilico_py.models import X10XeniumInput, DirectoryEntity, ExperimentViewSetting

Read in environmental variables.

In [10]:
# load environmental variables
load_dotenv(dotenv_path='../../.env')
load_dotenv(dotenv_path='../../.env.test')

True

In [9]:
def save_config(email, password):
    """
    Save a project config for the given email and password.
    """
    app_dir = typer.get_app_dir('cosilico_py')
    config_path: Path = Path(app_dir) / "config.json"
    d = {
        'api_url': os.environ.get('API_URL'),
        'anon_key': os.environ.get('ANON_KEY'),
        'cache_dir': '~/Downloads/temp_cache',
        'email': email,
        'password': password,
        'preprocessing': {
            'layer': {
                'cells_max_vert_map': {
                    1: 64,
                    2: 64,
                    4: 64,
                    8: 64,
                    16: 64,
                    32: 64,
                    64: 64,
                    128: 64,
                    256: 64,
                    512: 64,
                    1024: 64,
                    2048: 64,
                    4096: 64,
                    8192: 8,
                    16384: 8,
                    32768: 8,
                    65536: 8,
                    131072: 8,
                    262144: 8,
                    524288: 8,
                    1048576: 8,
                },
                'cells_downsample_map': {
                    1: -1,
                    2: -1,
                    4: -1,
                    8: -1,
                    16: -1,
                    32: -1,
                    64: -1,
                    128: -1,
                    256: -1,
                    512: -1,
                    1024: -1,
                    2048: -1,
                    4096: -1,
                    8192: 100_000,
                    16384: 100_000,
                    32768: 100_000,
                    65536: 100_000,
                    131072: 100_000,
                    262144: 100_000,
                    524288: 100_000,
                    1048576: 100_000,
                },
                'cells_object_type_map': {
                    1: 'polygon',
                    2: 'polygon',
                    4: 'polygon',
                    8: 'polygon',
                    16: 'polygon',
                    32: 'polygon',
                    64: 'polygon',
                    128: 'polygon',
                    256: 'polygon',
                    512: 'polygon',
                    1024: 'polygon',
                    2048: 'polygon',
                    4096: 'polygon',
                    8192: 'polygon',
                    16384: 'polygon',
                    32768: 'polygon',
                    65536: 'polygon',
                    131072: 'polygon',
                    262144: 'polygon',
                    524288: 'polygon',
                    1048576: 'polygon',
                }
            }
        }

    }

    Path(d['cache_dir']).mkdir(parents=True, exist_ok=True)


    json.dump(d, open(config_path, 'w'))

def autocreate_client(email, password):
    """
    Convenience method for creating client.
    """
    save_config(email, password)
    client = CosilicoClient()
    client.sign_in()
    return client
    
    

In [11]:
# supabase client
supabase = create_client(os.environ.get('API_URL'), os.environ.get('ANON_KEY'))
supabase

## Populate user

Creates a new user

In [15]:
response = supabase.auth.sign_up(
    {
        "email": os.environ.get('ME_EMAIL'),
        "password": os.environ.get('ME_PASSWORD'),
        'options': {
            'data': {
                'name': 'Erik Storrs',
            }
        }
    }
)
response

AuthResponse(user=User(id='605f1ad5-1079-44ed-929b-b9a33a51586a', app_metadata={'provider': 'email', 'providers': ['email']}, user_metadata={'email': 'epstorrs@gmail.com', 'email_verified': True, 'name': 'Erik Storrs', 'phone_verified': False, 'sub': '605f1ad5-1079-44ed-929b-b9a33a51586a'}, aud='authenticated', confirmation_sent_at=None, recovery_sent_at=None, email_change_sent_at=None, new_email=None, new_phone=None, invited_at=None, action_link=None, email='epstorrs@gmail.com', phone='', created_at=datetime.datetime(2025, 7, 28, 1, 32, 36, 300791, tzinfo=TzInfo(UTC)), confirmed_at=None, email_confirmed_at=datetime.datetime(2025, 7, 28, 1, 32, 36, 324476, tzinfo=TzInfo(UTC)), phone_confirmed_at=None, last_sign_in_at=datetime.datetime(2025, 7, 28, 1, 32, 36, 329249, tzinfo=TzInfo(UTC)), role='authenticated', updated_at=datetime.datetime(2025, 7, 28, 1, 32, 36, 335585, tzinfo=TzInfo(UTC)), identities=[UserIdentity(id='605f1ad5-1079-44ed-929b-b9a33a51586a', identity_id='ef7fdcaf-77bd-435

In [21]:
client = autocreate_client(os.environ.get('ME_EMAIL'), os.environ.get('ME_PASSWORD'))

Sign-in successful. 💥

## Create experiment

Creating some test directories

In [22]:
client.create_directory('/project_a/subproject_a/zzz', permission='rw')

Directory successfuly created at project_a/subproject_a/zzz

In [23]:
client.display_experiments()

Permission level 🔑: read write delete

└── 📁 project_a

└── 📁 subproject_a

└── 📁 zzz

Creating a 10X Xenium experiment from a spaceranger outputs folder

In [24]:
x_input = X10XeniumInput(
    cellranger_outs='/Users/erikstorrs/Downloads/Xenium_Prime_Breast_Cancer_FFPE_outs',
    bbox=(25000, 28000, 25000, 28000),
    to_uint8=True
)
x_input

X10XeniumInput(name=None, bbox=[25000, 28000, 25000, 28000], verbose=True, platform=<PlatformEnum.x10_xenium: '10X Xenium'>, cellranger_outs=PosixPath('/Users/erikstorrs/Downloads/Xenium_Prime_Breast_Cancer_FFPE_outs'), to_uint8=True)

In [25]:
bundle = client.create_experiment(x_input)

Loading xenium experiment from /Users/erikstorrs/Downloads/Xenium_Prime_Breast_Cancer_FFPE_outs

Loading xenium morphology image from 
/Users/erikstorrs/Downloads/Xenium_Prime_Breast_Cancer_FFPE_outs/morphology_focus/morphology_focus_0002.ome.tif

<tifffile.TiffFile 'morphology_focus_0002.ome.tif'> OME series cannot read multi-file pyramids


Loading xenium transcripts from 
/Users/erikstorrs/Downloads/Xenium_Prime_Breast_Cancer_FFPE_outs/transcripts.parquet

Loading xenium transcript metadata for ['qv']

/Users/erikstorrs/Documents/viewer/cosilico-py/src/cosilico_py/preprocessing/core/tiling.py:28: RuntimeWarning: divide by zero encountered in remainder
  groups = np.arange(num_feats) % group_size  # Vectorized operation


Loading xenium cell boundaries from 
/Users/erikstorrs/Downloads/Xenium_Prime_Breast_Cancer_FFPE_outs/cell_boundaries.parquet

Loading xenium cell transcript counts 
/Users/erikstorrs/Downloads/Xenium_Prime_Breast_Cancer_FFPE_outs/cell_feature_matrix.h5

Uploading the experiment

In [26]:
client.upload_experiment(bundle, '/project_a/subproject_a/zzz')

Experiment Xenium Prime + 100g Custom-Breast_Cancer successfuly uploaded 🔬!

In [27]:
client.display_experiments()

Permission level 🔑: read write delete

└── 📁 project_a

└── 📁 subproject_a

└── 📁 zzz

└── 🔬 Xenium Prime + 100g Custom-Breast_Cancer

## Downloading and uploading additional data

We can also download the experiment and extract the image and layer data

In [29]:
experiment = client.get_experiment('/project_a/subproject_a/zzz/Xenium Prime + 100g Custom-Breast_Cancer')

In [30]:
experiment.images ## The IHC image

[Image(id='d971a1e5-d441-4a9e-8e05-79da74b4e3cc', version='v0.0.1', experiment_id='429ed69f-28e9-4663-8e71-222a7fbc7533', name='Xenium Morphology', metadata=OME(
    plates=[{'id': 'Plate:0', 'well_origin_x': 0.0, 'well_origin_x_unit': <UnitsLength.MICROMETER: 'µm'>, 'well_origin_y': 0.0, 'well_origin_y_unit': <UnitsLength.MICROMETER: 'µm'>}],
    instruments=[{'microscope': {'manufacturer': '10x Genomics', 'model': 'Xenium', 'kind': 'microscope'}, 'id': 'Instrument:0'}],
    images=[<1 field_type>],
    structured_annotations={'map_annotations': [{'id': 'Annotation:0', 'value': {'Purpose': 'Nuclear'}, 'kind': 'mapannotation'}, {'id': 'Annotation:1', 'value': {'Purpose': 'Boundary'}, 'kind': 'mapannotation'}, {'id': 'Annotation:2', 'value': {'Purpose': 'Interior - RNA'}, 'kind': 'mapannotation'}, {'id': 'Annotation:3', 'value': {'Purpose': 'Interior - Protein'}, 'kind': 'mapannotation'}]},
    uuid='urn:uuid:fddd3688-7c75-11ef-a04f-067698406db1',
 ), tags=[], local_path='/Users/eriksto

Reading the image as a dask array.

In [33]:
image = experiment.images[0]
arr = experiment.generate_image_data(image) # image is of shape XYZCT
arr

dask.array<reshape, shape=(3072, 3072, 1, 4, 1), dtype=uint8, chunksize=(512, 512, 1, 1, 1), chunktype=numpy.ndarray>

We can also extract data of a given layer

In [38]:
layer = experiment.get_layer('Transcripts')
df = experiment.generate_layer_data(layer)
df

,x_location,y_location,feature_name,counts,QV
id,,,,,
282406984613900,681.101562,2080.072266,STIM1,1.0,31.00
282406984613956,684.410156,1912.500000,RARA,1.0,31.25
282406984613958,687.867188,2239.851562,HUWE1,1.0,40.00
282406984613969,678.750000,528.013672,CD300C,1.0,40.00
282406984613979,688.013672,2141.910156,PAK1IP1,1.0,34.75
...,...,...,...,...,...
282548719750377,720.146484,2405.146484,SOD1,1.0,40.00
282548719750387,728.529297,2715.367188,FLII,1.0,40.00
282548719750395,731.029297,2417.205078,TMEM106B,1.0,40.00


For grouped layers, we can generate an AnnData object to store the sparse matrix-based data

In [39]:
layer = experiment.get_layer('Cells')
adata = experiment.generate_layer_data(layer, experiment.get_layer_metadata(layer, metadata_name='Transcript Counts'))
adata

/Users/erikstorrs/Documents/viewer/cosilico-py/src/cosilico_py/client/experiment.py:206: ImplicitModificationWarning: Setting element `.obsm['spatial']` of view, initializing view as actual.
  adata.obsm['spatial'] = df[['x_location', 'y_location']].values


View of AnnData object with n_obs × n_vars = 3100 × 5099
    obsm: 'spatial'

Here we add new K means clusters to the cell layer to demonstrate uploading categorical data

In [40]:
a = adata.copy()
sc.pp.normalize_total(a)
sc.pp.log1p(a)
sc.pp.pca(a)

n_clusters = 10
x = KMeans(n_clusters=n_clusters).fit_transform(a.obsm['X_pca'])
clusters = x.argmax(1)
adata.obs['Kmeans N=10'] = [f'cluster {x}' for x in clusters]
adata.obs

/var/folders/l4/tr_3w7sx31z58xhsnb0_07hw0000gr/T/ipykernel_52975/2572132737.py:9: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs['Kmeans N=10'] = [f'cluster {x}' for x in clusters]


,Kmeans N=10
bbeodhla-1,cluster 9
bbeoeddl-1,cluster 9
bbeognab-1,cluster 9
bbeognch-1,cluster 9
bbeoickk-1,cluster 9
...,...
oanleihg-1,cluster 6
oanlfaan-1,cluster 0
oanmapng-1,cluster 0
oanmbbjc-1,cluster 6


In [41]:
experiment.create_categorical_layer_metadata('Kmeans N=10', adata.obs['Kmeans N=10'], layer)

LayerMetadata(id='55a6d06f65bf45e3a4577308da73066c', version='v0.0.1', layer_id='6856bd60-108c-4a8d-b439-46d85f089aaf', name='Kmeans N=10', metadata_type=<MetadataTypeEnum.categorical: 'categorical'>, is_sparse=False, fields=['cluster 0', 'cluster 1', 'cluster 2', 'cluster 3', 'cluster 5', 'cluster 6', 'cluster 7', 'cluster 8', 'cluster 9'], metadata={}, tags=[], local_path=PosixPath('/Users/erikstorrs/Downloads/temp_cache/55a6d06f65bf45e3a4577308da73066c.zarr.zip'), path='55a6d06f65bf45e3a4577308da73066c.zarr.zip')

Here we add transcript counts for each gene to the cell layer to demonstrate uploading continuous data

In [43]:
layer = experiment.get_layer('Cells')
adata = experiment.generate_layer_data(layer, experiment.get_layer_metadata(layer, metadata_name='Transcript Counts'))
adata

/Users/erikstorrs/Documents/viewer/cosilico-py/src/cosilico_py/client/experiment.py:206: ImplicitModificationWarning: Setting element `.obsm['spatial']` of view, initializing view as actual.
  adata.obsm['spatial'] = df[['x_location', 'y_location']].values


AnnData object with n_obs × n_vars = 3100 × 5099
    obs: 'Kmeans N=10'
    obsm: 'spatial'

In [45]:
pca_df = pd.DataFrame(a.obsm['X_pca'][:, :10], columns=[f'PCA {i}' for i in range(10)], index=a.obs.index)
pca_df

,PCA 0,PCA 1,PCA 2,PCA 3,PCA 4,PCA 5,PCA 6,PCA 7,PCA 8,PCA 9
bbeodhla-1,1.884373,0.019806,-0.535612,-0.923417,0.284337,0.008906,-0.136600,-0.334375,-0.140023,0.341846
bbeoeddl-1,1.265127,-0.323408,-0.310701,0.686867,0.036360,-0.192906,-0.659026,-0.315289,-0.464223,0.211977
bbeognab-1,2.074790,0.245756,0.023433,-0.204142,0.301107,-0.060960,0.208282,-0.406961,0.383027,-0.363321
bbeognch-1,0.513875,0.093370,0.638010,-1.171560,-2.071590,-1.096752,1.004826,-0.249507,1.815500,0.557605
bbeoickk-1,1.887420,-0.376451,-0.484315,-0.783921,0.303812,0.477196,0.031322,-0.487668,-0.020544,-0.508676
...,...,...,...,...,...,...,...,...,...,...
oanleihg-1,-3.184845,-0.481612,0.098723,-1.368790,2.467744,-1.680421,0.998374,0.083817,-0.362879,-0.378304
oanlfaan-1,-3.321548,-0.629462,0.137387,-1.046022,0.259058,-0.435915,0.135748,0.049865,-0.061714,-0.677603
oanmapng-1,-1.624093,-1.117543,-0.568650,-1.149179,-1.911695,-0.569102,0.430738,-0.210436,-0.385713,0.736338
oanmbbjc-1,-2.271970,-0.928231,-0.115664,-1.440908,-1.031893,-0.359986,-0.584330,-0.560303,-0.427061,-0.787605


In [46]:
experiment.add_continuous_layer_metadata('PCAs', pca_df, layer)

LayerMetadata(id='ea3f6cff73e44d37b03153f08985212c', version='v0.0.1', layer_id='6856bd60-108c-4a8d-b439-46d85f089aaf', name='PCAs', metadata_type=<MetadataTypeEnum.continuous: 'continuous'>, is_sparse=False, fields=['PCA 0', 'PCA 1', 'PCA 2', 'PCA 3', 'PCA 4', 'PCA 5', 'PCA 6', 'PCA 7', 'PCA 8', 'PCA 9'], metadata={}, tags=[], local_path=PosixPath('/Users/erikstorrs/Downloads/temp_cache/ea3f6cff73e44d37b03153f08985212c.zarr.zip'), path='ea3f6cff73e44d37b03153f08985212c.zarr.zip')

Check out the resulting experiment [here](https://lighthearted-kulfi-ce56ba.netlify.app/portal/demo_429ed69f-28e9-4663-8e71-222a7fbc7533).